## Callables can take callables...

In [216]:
def hello(printer):
    printer("Hello world")

hello(print)

def exclamation(s):
    print(f"{s}!")

hello(exclamation)

Hello world
Hello world!


## ...and return callables!

In [220]:
def greeter(printer):
    def greet(name):
        # Inner function has - and KEEPS - the context in which is was 
        # created ("closes over", aka "closure")
        printer(f"Hello {name}")
    return greet

hi = greeter(print)
hi_bang = greeter(exclamation)

hi("Python Study Group")
hi_bang("Python Study Group")

Hello Python Study Group
Hello Python Study Group!


## Let's try this again

In [233]:
from random import choice
def flaky():
    return choice(["Okay", None])
    
flaky()

'Okay'

In [235]:
def retry(f):
    tries = 5
    for t in range(tries):
        result = f()
        if result:
            return result
        print(f"fail {t}")
    return result
retry(flaky)

fail 0
fail 1
fail 2
fail 3


'Okay'

In [236]:
def retried(f):
    tries = 5
    def retry():
        for t in range(tries):
            result = f()
            if result:
                return result
            print(f"fail {t}")
        return result
    return retry

less_flaky = retried(flaky)
less_flaky()

fail 0


'Okay'

## Add a little syntactic sugar...

In [167]:
@retried
def flaky():
    return choice(["Okay", None])

flaky()

fail 0
fail 1
fail 2
fail 3


'Okay'

## How does that work?

**retried**: expression that _<u>takes</u>_ a callable and _<u>returns</u>_ a callable that _<u>replaces<u>_ the following one

In [179]:
retried

<function __main__.retried(f)>

**@**: the sugar, automatically calls the decorator _<u>at declaration time<u>_ with the following one and does the replacement, basically like this:

```python
flaky = retried(flaky)
```

## Getting fancy

Let's make a version with configurable tries...

...but wait, if the `@` automatically calls it with the following thing - _<u>just</u>_ the following thing - where would arguments for _<u>it</u>_ go?

## Inception

Callables can return _<u>generators!</u>_